In [ ]:
!pip install gradio transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr
import re

model_path = "prajjwal888/Llama-2-7b-chat-question-generation"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
def parse_generated_text(text: str) -> dict:
    clean_text = re.sub(r"\[/?INST\]", "", text)
    clean_text = re.sub(r"Question:\s*Question:", "Question:", clean_text)
    clean_text = clean_text.strip()

    match = re.search(r"Question:\s*(.*?)(?:\nHint:|Hint:)(.*)", clean_text, re.DOTALL)

    if match:
        question = match.group(1).strip().strip('"').replace("Question:", "").strip()
        hint = match.group(2).strip().strip('"')
    else:
        question = clean_text.strip()
        hint = "No hint available"

    return {
        "question": question,
        "hint": hint
    }

def generate_questions(topic, difficulty, types, count):
    print("Received input:", topic, difficulty, types, count)

    try:
        pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            # device=0,
            max_length=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        questions = []

        for _ in range(count):
            for q_type in types:
                prompt = (
                    f"Generate a {difficulty} difficulty {q_type} question about {topic}.\n"
                    "Format strictly as follows:\n"
                    "Question: <your question here>\n"
                    "Hint: <your hint here or 'No hint available'>"
                )

                formatted_prompt = f"<s>[INST] {prompt} [/INST]"
                print("Prompt:", formatted_prompt)

                result = pipe(formatted_prompt)
                print("Raw Output:", result)

                generated_text = result[0]['generated_text'].replace(formatted_prompt, "").strip()
                parsed = parse_generated_text(generated_text)

                questions.append(f"**Type**: {q_type}\n\n**Question**: {parsed['question']}\n\n**Hint**: {parsed['hint']}\n\n---")

        return "\n\n".join(questions)

    except Exception as e:
        print("Error:", e)
        return f"Something went wrong: {e}"

iface = gr.Interface(
    fn=generate_questions,
    inputs=[
        gr.Textbox(label="Topic"),
        gr.Dropdown(choices=["easy", "medium", "hard"], label="Difficulty", value="medium"),
        gr.CheckboxGroup(choices=["Conceptual", "Numerical", "Application"], label="Question Types"),
        gr.Slider(minimum=1, maximum=5, step=1, value=2, label="Number of Questions per Type")
    ],
    outputs=gr.Markdown(label="Generated Questions"),
    title="AI Question Generator",
    description="Enter a topic, select difficulty and question types to generate AI-powered questions."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://01920a9c15f57ed481.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr
import re

model_path = "prajjwal888/Llama-2-7b-chat-question-generation"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

def parse_generated_text(text: str) -> dict:
    clean_text = re.sub(r"\[/?INST\]", "", text)
    clean_text = re.sub(r"Question:\s*Question:", "Question:", clean_text)
    clean_text = clean_text.strip()

    match = re.search(r"Question:\s*(.*?)(?:\nHint:|Hint:)(.*)", clean_text, re.DOTALL)

    if match:
        question = match.group(1).strip().strip('"').replace("Question:", "").strip()
        hint = match.group(2).strip().strip('"')
    else:
        question = clean_text.strip()
        hint = "No hint available"

    return {
        "question": question,
        "hint": hint
    }

def generate_questions(topic, difficulty, types, count):
    print("Received input:", topic, difficulty, types, count)

    try:
        pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            # device=0,
            max_length=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        questions = []

        for _ in range(count):
            for q_type in types:
                prompt = (
                    f"Generate a {difficulty} difficulty {q_type} question about {topic}.\n"
                    "Format strictly as follows:\n"
                    "Question: <your question here>\n"
                    "Hint: <your hint here or 'No hint available'>"
                )

                formatted_prompt = f"<s>[INST] {prompt} [/INST]"
                print("Prompt:", formatted_prompt)

                result = pipe(formatted_prompt)
                print("Raw Output:", result)

                generated_text = result[0]['generated_text'].replace(formatted_prompt, "").strip()
                parsed = parse_generated_text(generated_text)

                questions.append(f"**Type**: {q_type}\n\n**Question**: {parsed['question']}\n\n**Hint**: {parsed['hint']}\n\n---")

        return "\n\n".join(questions)

    except Exception as e:
        print("Error:", e)
        return f"Something went wrong: {e}"

iface = gr.Interface(
    fn=generate_questions,
    inputs=[
        gr.Textbox(label="Topic"),
        gr.Dropdown(choices=["easy", "medium", "hard"], label="Difficulty", value="medium"),
        gr.CheckboxGroup(choices=["Conceptual", "Numerical", "Application"], label="Question Types"),
        gr.Slider(minimum=1, maximum=5, step=1, value=2, label="Number of Questions per Type")
    ],
    outputs=gr.Markdown(label="Generated Questions"),
    title="AI Question Generator",
    description="Enter a topic, select difficulty and question types to generate AI-powered questions."
)

iface.queue()
iface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84d8135c05fbf778f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 